In [1]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision.models import resnet50, ResNet50_Weights
from torchvision.transforms import Resize, InterpolationMode, ToPILImage
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import WandbLogger
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
import torch
import cv2
import wandb
import argparse
import os
import random

import pandas as pd
import cv2
import numpy as np
import albumentations as A

In [2]:
class NeoPolypDataset(Dataset):
    def __init__(
        self,
        image_dir: list,
        gt_dir: list | None = None,
        session: str = "train"
    ) -> None:
        super().__init__()
        self.session = session
        if session == "train":
            self.train_path = image_dir
            self.train_gt_path = gt_dir
            self.len = len(self.train_path)
            self.train_transform = TrainTransform()
        elif session == "val":
            self.val_path = image_dir
            self.val_gt_path = gt_dir
            self.len = len(self.val_path)
            self.val_transform = ValTransform()
        else:
            self.test_path = image_dir
            self.len = len(self.test_path)
            self.test_transform = TestTransform()
            
    @staticmethod
    def _read_mask(mask_path):
        image = cv2.imread(mask_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        # lower boundary RED color range values; Hue (0 - 10)
        lower1 = np.array([0, 100, 20])
        upper1 = np.array([10, 255, 255])
        # upper boundary RED color range values; Hue (160 - 180)
        lower2 = np.array([160, 100, 20])
        upper2 = np.array([179, 255, 255])
        lower_mask = cv2.inRange(image, lower1, upper1)
        upper_mask = cv2.inRange(image, lower2, upper2)

        red_mask = lower_mask + upper_mask
        red_mask[red_mask != 0] = 1

        # boundary GREEN color range values; Hue (36 - 70)
        green_mask = cv2.inRange(image, (36, 25, 25), (70, 255, 255))
        green_mask[green_mask != 0] = 2

        full_mask = cv2.bitwise_or(red_mask, green_mask)
        full_mask = full_mask.astype(np.uint8)
        return full_mask

    def __len__(self) -> int:
        return self.len

    def __getitem__(self, index: int):
        if self.session == "train":
            img = cv2.imread(self.train_path[index])
            gt = self._read_mask(self.train_gt_path[index])
            return self.train_transform(img, gt)
        elif self.session == "val":
            img = cv2.imread(self.val_path[index])
            gt = self._read_mask(self.val_gt_path[index])
            return self.val_transform(img, gt)
        else:
            img = cv2.imread(self.test_path[index])
            H, W, _ = img.shape
            img = self.test_transform(img)
            file_id = self.test_path[index].split('/')[-1].split('.')[0]
            return img, file_id, H, W

In [3]:
class TrainTransform:
    def __init__(self) -> None:
        self.transform = A.Compose([
            A.HorizontalFlip(p=0.3),
            A.VerticalFlip(p=0.3),
            A.RandomGamma(gamma_limit=(70, 130), always_apply=False, p=0.2), #TypeError: RandomGamma.__init__() got an unexpected keyword argument 'eps' --> deleted eps=None
            A.RGBShift(p=0.3, r_shift_limit=10, g_shift_limit=10, b_shift_limit=10),
            A.OneOf([A.Blur(), A.GaussianBlur(), A.GlassBlur(), A.MotionBlur(),
                    A.GaussNoise(), A.Sharpen(), A.MedianBlur(), A.MultiplicativeNoise()]),
            A.CoarseDropout(p=0.2, max_height=35, max_width=35, fill_value=255),
            A.RandomSnow(snow_point_lower=0.1, snow_point_upper=0.15, brightness_coeff=1.5, p=0.09),
            A.RandomShadow(p=0.1),
            A.ShiftScaleRotate(p=0.45, border_mode=cv2.BORDER_CONSTANT, shift_limit=0.15, scale_limit=0.15),
            A.Resize(256, 256, interpolation=cv2.INTER_LINEAR),
            A.Normalize(),
            ToTensorV2(),
        ])

    def __call__(self, img, mask):
        return self.transform(image=img, mask=mask)


class ValTransform:
    def __init__(self) -> None:
        self.transform = A.Compose([
            A.Resize(256, 256, interpolation=cv2.INTER_LINEAR),
            A.Normalize(),
            ToTensorV2(),
        ])

    def __call__(self, img, mask):
        return self.transform(image=img, mask=mask)


class TestTransform:
    def __init__(self) -> None:
        self.transform = A.Compose([
            A.Resize(256, 256, interpolation=cv2.INTER_LINEAR),
            A.Normalize(),
            ToTensorV2(),
        ])

    def __call__(self, img):
        return self.transform(image=img)['image']

In [4]:
#LOSS


def mask2rgb(mask):
    color_dict = {0: torch.tensor([0, 0, 0]),
                  1: torch.tensor([1, 0, 0]),
                  2: torch.tensor([0, 1, 0])}
    output = torch.zeros((mask.shape[0], mask.shape[1], mask.shape[2], 3)).long()
    for i in range(mask.shape[0]):
        for k in color_dict.keys():
            output[i][mask[i].long() == k] = color_dict[k]
    return output.to(mask.device)


@torch.no_grad()
def dice_score(
    inputs: torch.Tensor,
    targets: torch.Tensor
) -> torch.Tensor:
    # compute softmax over the classes axis
    input_one_hot = mask2rgb(inputs.argmax(dim=1))

    # create the labels one hot tensor
    target_one_hot = mask2rgb(targets)

    # compute the actual dice score
    dims = (2, 3)
    intersection = torch.sum(input_one_hot * target_one_hot, dims)
    cardinality = torch.sum(input_one_hot + target_one_hot, dims)

    dice_score = (2. * intersection + 1e-6) / (cardinality + 1e-6)
    return dice_score.mean()


class DiceLoss(nn.Module):
    def __init__(self, weights=torch.Tensor([[0.4, 0.55, 0.05]])) -> None:
        super(DiceLoss, self).__init__()
        self.eps = 1e-6
        self.weights = weights

    def forward(
            self,
            inputs: torch.Tensor,
            targets: torch.Tensor) -> torch.Tensor:
        input_soft = F.softmax(inputs, dim=1)

        target_one_hot = mask2rgb(targets)

        dims = (2, 3)
        intersection = torch.sum(input_soft * target_one_hot, dims)
        cardinality = torch.sum(input_soft + target_one_hot, dims)

        dice_score = 2. * intersection / (cardinality + self.eps)

        dice_score = torch.sum(
            dice_score * self.weights.to(dice_score.device),
            dim=1
        )
        return torch.mean(1. - dice_score)

In [5]:
#UNET
def _make_layers(in_channels: int, out_channels: int):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, bias=False),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, bias=False),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True)
    )


class RecurrentBlock(nn.Module):
    def __init__(self, ch_out, t=2):
        super(RecurrentBlock, self).__init__()
        self.t = t
        self.ch_out = ch_out
        self.conv = nn.Sequential(
            nn.Conv2d(ch_out, ch_out, kernel_size=3, stride=1, padding=1, bias=True),
            nn.BatchNorm2d(ch_out),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        for i in range(self.t):

            if i == 0:
                x1 = self.conv(x)

            x1 = self.conv(x+x1)
        return x1


class R2Block(nn.Module):
    def __init__(self, ch_in, ch_out, t=2, max_pool=True):
        super(R2Block, self).__init__()
        self.pool = max_pool
        if max_pool:
            self.max_pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.RCNN = nn.Sequential(
            RecurrentBlock(ch_out, t=t),
            RecurrentBlock(ch_out, t=t)
        )
        self.Conv_1x1 = nn.Conv2d(ch_in, ch_out, kernel_size=1, stride=1, padding=0)

    def forward(self, x):
        if self.pool:
            x = self.max_pool(x)
        x = self.Conv_1x1(x)
        x1 = self.RCNN(x)
        return x+x1


class Attention(nn.Module):
    def __init__(self, F_g, F_l, F_int):
        super(Attention, self).__init__()
        self.W_g = nn.Sequential(
            nn.Conv2d(F_g, F_int, 1, 1, 0, bias=True),
            nn.BatchNorm2d(F_int)
        )
        self.W_x = nn.Sequential(
            nn.Conv2d(F_l, F_int, 1, 1, 0, bias=True),
            nn.BatchNorm2d(F_int)
        )
        self.psi = nn.Sequential(
            nn.Conv2d(F_int, 1, 1, 1, 0, bias=True),
            nn.BatchNorm2d(1),
            nn.Sigmoid()
        )
        self.relu = nn.ReLU(inplace=True)

    def forward(self, g, x):
        g1 = self.W_g(g)
        x1 = self.W_x(x)
        psi = self.relu(g1+x1)
        psi = self.psi(psi)

        return x*psi


class DownSample(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.down = nn.Sequential(
            nn.MaxPool2d(kernel_size=2, stride=2),
            _make_layers(in_channels, out_channels)
        )

    def forward(self, x):
        return self.down(x)


class UpSample(nn.Module):
    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        attention: bool = False,
        recurrent: bool = True
    ):
        super().__init__()
        self.attention = attention
        self.up_conv = nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2)
        if attention:
            self.attn = Attention(out_channels, out_channels, out_channels//2)
        if recurrent:
            self.conv = R2Block(in_channels, out_channels, max_pool=False)
        else:
            self.conv = _make_layers(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up_conv(x1)
        if self.attention:
            x2 = self.attn(x1, x2)
        out = torch.cat([x2, x1], dim=1)
        out = self.conv(out)
        return out


class UNet(nn.Module):
    def __init__(
        self,
        in_channels: int,
        attention: bool = True,
        recurrent: bool = True
    ):
        super().__init__()
        self.attention = attention

        if recurrent:
            self.conv_in = R2Block(in_channels, 64, max_pool=False)
            self.down1 = R2Block(64, 128)
            self.down2 = R2Block(128, 256)
            self.down3 = R2Block(256, 512)
            self.down4 = R2Block(512, 1024)
        else:
            self.conv_in = _make_layers(in_channels, 64)
            self.down1 = DownSample(64, 128)
            self.down2 = DownSample(128, 256)
            self.down3 = DownSample(256, 512)
            self.down4 = DownSample(512, 1024)

        self.up1 = UpSample(1024, 512, attention=attention, recurrent=recurrent)
        self.up2 = UpSample(512, 256, attention=attention, recurrent=recurrent)
        self.up3 = UpSample(256, 128, attention=attention, recurrent=recurrent)
        self.up4 = UpSample(128, 64, attention=attention, recurrent=recurrent)
        self.conv_out = nn.Conv2d(64, 3, kernel_size=1)

    def forward(self, x):
        x1 = self.conv_in(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x = self.down4(x4)
        x = self.up1(x, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        x = self.conv_out(x)
        return x

In [6]:
#MODEL

class NeoPolypModel(pl.LightningModule):
    def __init__(self, lr: float = 1e-4, name: str = "resunet"):
        super().__init__()
        if name == "resunet":
            self.model = Resnet50Unet(n_classes=3)
        else:
            self.model = UNet(in_channels=3)
        self.lr = lr
        self.dice_loss = DiceLoss()
        self.entropy_loss = nn.CrossEntropyLoss()

    def forward(self, x):
        return self.model(x)

    def _forward(self, batch, batch_idx, name="train"):
        image, mask = batch['image'].float(), batch['mask'].long()
        logits = self(image)
        loss = self.entropy_loss(logits, mask)
        d_score = dice_score(logits, mask)
        acc = (logits.argmax(dim=1) == mask).float().mean()
        self.log_dict(
            {
                f"{name}_loss": loss,
                f"{name}_dice_score": d_score,
                f"{name}_acc": acc
            },
            on_step=False, on_epoch=True, sync_dist=True, prog_bar=True
        )
        return loss

    def training_step(self, batch, batch_idx):
        return self._forward(batch, batch_idx, "train")

    def validation_step(self, batch, batch_idx):
        return self._forward(batch, batch_idx, "val")

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(
            params=self.parameters(),
            lr=self.lr,
        )
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer=optimizer,
            patience=5,
            verbose=True,
            factor=0.5
        )
        return {
            'optimizer': optimizer,
            'lr_scheduler': scheduler,
            'monitor': 'val_loss',
            'interval': 'epoch',
        }

In [7]:
#RESUNET

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, padding=1, kernel_size=3, stride=1, with_nonlinearity=True):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, padding=padding, kernel_size=kernel_size, stride=stride)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
        self.with_nonlinearity = with_nonlinearity

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        if self.with_nonlinearity:
            x = self.relu(x)
        return x


class Bridge(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.bridge = nn.Sequential(
            ConvBlock(in_channels, out_channels),
            ConvBlock(out_channels, out_channels)
        )

    def forward(self, x):
        return self.bridge(x)


class UpsampleBlock(nn.Module):

    def __init__(self, in_channels, out_channels, up_conv_in_channels=None, up_conv_out_channels=None,
                 upsampling_method="conv_transpose"):
        super().__init__()

        if up_conv_in_channels is None:
            up_conv_in_channels = in_channels
        if up_conv_out_channels is None:
            up_conv_out_channels = out_channels

        if upsampling_method == "conv_transpose":
            self.upsample = nn.ConvTranspose2d(up_conv_in_channels, up_conv_out_channels, kernel_size=2, stride=2)
        elif upsampling_method == "bilinear":
            self.upsample = nn.Sequential(
                nn.Upsample(mode='bilinear', scale_factor=2),
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1)
            )
        self.conv_block_1 = ConvBlock(in_channels, out_channels)
        self.conv_block_2 = ConvBlock(out_channels, out_channels)

    def forward(self, up_x, down_x):
        x = self.upsample(up_x)
        x = torch.cat([x, down_x], 1)
        x = self.conv_block_1(x)
        x = self.conv_block_2(x)
        return x


class Resnet50Unet(nn.Module):
    DEPTH = 6

    def __init__(self, n_classes=2):
        super().__init__()
        resnet = resnet50(weights=ResNet50_Weights.DEFAULT)
        down_blocks = []
        up_blocks = []
        self.input_block = nn.Sequential(*list(resnet.children()))[:3]
        self.input_pool = list(resnet.children())[3]
        for bottleneck in list(resnet.children()):
            if isinstance(bottleneck, nn.Sequential):
                down_blocks.append(bottleneck)
        self.down_blocks = nn.ModuleList(down_blocks)
        self.bridge = Bridge(2048, 2048)
        up_blocks.append(UpsampleBlock(2048, 1024))
        up_blocks.append(UpsampleBlock(1024, 512))
        up_blocks.append(UpsampleBlock(512, 256))
        up_blocks.append(UpsampleBlock(in_channels=128 + 64, out_channels=128,
                                       up_conv_in_channels=256, up_conv_out_channels=128))
        up_blocks.append(UpsampleBlock(in_channels=64 + 3, out_channels=64,
                                       up_conv_in_channels=128, up_conv_out_channels=64))

        self.up_blocks = nn.ModuleList(up_blocks)

        self.out = nn.Conv2d(64, n_classes, kernel_size=1, stride=1)

    def forward(self, x, with_output_feature_map=False):
        pre_pools = dict()
        pre_pools["layer_0"] = x
        x = self.input_block(x)
        pre_pools["layer_1"] = x
        x = self.input_pool(x)

        for i, block in enumerate(self.down_blocks, 2):
            x = block(x)
            if i == (Resnet50Unet.DEPTH - 1):
                continue
            pre_pools[f"layer_{i}"] = x

        x = self.bridge(x)

        for i, block in enumerate(self.up_blocks, 1):
            key = f"layer_{Resnet50Unet.DEPTH - 1 - i}"
            x = block(x, pre_pools[key])
        output_feature_map = x
        x = self.out(x)
        del pre_pools
        if with_output_feature_map:
            return x, output_feature_map
        else:
            return x

In [8]:
# DATALOADER
image_path = []
TRAIN_DIR = '/kaggle/input/bkai-igh-neopolyp/train/train'
for root, dirs, files in os.walk(TRAIN_DIR):
    # iterate over 1000 images
    for file in files:
        # create path
        path = os.path.join(root,file)
        # add path to list
        image_path.append(path) 
mask_path = []
TRAIN_MASK_DIR = '/kaggle/input/bkai-igh-neopolyp/train_gt/train_gt'
for root, dirs, files in os.walk(TRAIN_MASK_DIR):
    #iterate over 1000 masks
    for file in files:
        # obtain the path"
        path = os.path.join(root,file)
        # add path to the list
        mask_path.append(path)
len(mask_path)

shuffle_list = list(zip(image_path, mask_path))
random.shuffle(shuffle_list)
image_path, mask_path = zip(*shuffle_list)

train_size = int(0.9 * len(image_path))
train_path = image_path[:train_size]
train_gt_path = mask_path[:train_size]
val_path = image_path[train_size:]
val_gt_path = mask_path[train_size:]
train_dataset = NeoPolypDataset(train_path, train_gt_path, session="train")
val_dataset = NeoPolypDataset(val_path, val_gt_path, session="val")

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=8,
    num_workers=4,
    shuffle=True
)

val_loader = DataLoader(
    dataset=val_dataset,
    batch_size=8,
    num_workers=4,
    shuffle=False
)

In [9]:
#WANDB
wandb.login(
    # set the wandb project where this run will be logged
#     project= "PolypSegment", 
    key = "6655aabde85f03a330ed57106c23cbd8390e2ae1",
)
wandb.init(
    project = "PolypSegment"
)
name = "unet"
logger = WandbLogger(project="PolypSegment",
                     name=name,
                     log_model="all")
# MODEL
model = NeoPolypModel(lr=0.0001)

# CALLBACK
root_path = os.path.join(os.getcwd(), "checkpoints")
ckpt_path = os.path.join(os.path.join(root_path, "model/"))
if not os.path.exists(root_path):
    os.makedirs(root_path)
if not os.path.exists(ckpt_path):
    os.makedirs(ckpt_path)

ckpt_callback = ModelCheckpoint(
    monitor="val_dice_score",
    dirpath=ckpt_path,
    filename="model",
    save_top_k=1,
    mode="max"
)  # save top 2 epochs with the highest val_dice_score
lr_callback = LearningRateMonitor("step")

early_stop_callback = EarlyStopping(
    monitor="val_loss",
    patience=15,
    verbose=True,
    mode="min"
)

# TRAINER
trainer = pl.Trainer(
    default_root_dir=root_path,
    logger=logger,
    callbacks=[
        ckpt_callback, lr_callback, early_stop_callback
    ],
    gradient_clip_val=1.0,
    max_epochs=200,
    enable_progress_bar=True,
    deterministic=False,
    accumulate_grad_batches=1
)

# FIT MODEL
trainer.fit(model=model,
            train_dataloaders=train_loader,
            val_dataloaders=val_loader)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: vuhoangphuc0000. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.17.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.16.6
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240517_133944-h3h19ss7
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run misty-butterfly-7
wandb: ⭐️ View project at https://wandb.ai/vuhoangphuc0000/PolypSegment
wandb: 🚀 View run at https://wandb.ai/vuhoangphuc0000/PolypSegment/runs/h3h19ss7
Downloading: "https://download.pytorch.org/models/resnet50-11a

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 00042: reducing learning rate of group 0 to 1.2500e-05.


Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [10]:
def mask2rgb(mask):
    color_dict = {0: torch.tensor([0, 0, 0]),
                  1: torch.tensor([1, 0, 0]),
                  2: torch.tensor([0, 1, 0])}
    output = torch.zeros((mask.shape[0], mask.shape[1], 3)).long()
    for k in color_dict.keys():
        output[mask.long() == k] = color_dict[k]
    return output.to(mask.device)

In [11]:
model = NeoPolypModel.load_from_checkpoint('checkpoints/model/model.ckpt')
model.eval()
test_path = []
TEST_DIR = '/kaggle/input/bkai-igh-neopolyp/test/test/'
for root, dirs, files in os.walk(TEST_DIR):
    for file in files:
        # create path
        path = os.path.join(root,file)
        # add path to list
        test_path.append(path)
test_dataset = NeoPolypDataset(test_path, session="test")
test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=1,
    num_workers=0,
    shuffle=False
)
if not os.path.isdir('/kaggle/working/predicted_masks'):
    os.mkdir('/kaggle/working/predicted_masks')
for _, (img, file_id, H, W) in enumerate(tqdm(test_dataloader, total=len(test_dataloader))):
    with torch.no_grad():
        predicted_mask = model(img.cuda())
    for i in range(1):
        filename = file_id[i] + ".png"
        argmax = torch.argmax(predicted_mask[i], 0)
        one_hot = mask2rgb(argmax).float().permute(2, 0, 1)
        mask2img = Resize((H[i].item(), W[i].item()), interpolation=InterpolationMode.NEAREST)(
            ToPILImage()(one_hot))
        mask2img.save(os.path.join('/kaggle/working/predicted_masks', filename))

100%|██████████| 200/200 [00:23<00:00,  8.39it/s]


In [12]:
def rle_to_string(runs):
    return ' '.join(str(x) for x in runs)

def rle_encode_one_mask(mask):
    pixels = mask.flatten()
    pixels[pixels > 0] = 255
    use_padding = False
    if pixels[0] or pixels[-1]:
        use_padding = True
        pixel_padded = np.zeros([len(pixels) + 2], dtype=pixels.dtype)
        pixel_padded[1:-1] = pixels
        pixels = pixel_padded
    
    rle = np.where(pixels[1:] != pixels[:-1])[0] + 2
    if use_padding:
        rle = rle - 1
    rle[1::2] = rle[1::2] - rle[:-1:2]
    return rle_to_string(rle)

def mask2string(dir):
    ## mask --> string
    strings = []
    ids = []
    ws, hs = [[] for i in range(2)]
    for image_id in os.listdir(dir):
        id = image_id.split('.')[0]
        path = os.path.join(dir, image_id)
        print(path)
        img = cv2.imread(path)[:,:,::-1]
        h, w = img.shape[0], img.shape[1]
        for channel in range(2):
            ws.append(w)
            hs.append(h)
            ids.append(f'{id}_{channel}')
            string = rle_encode_one_mask(img[:,:,channel])
            strings.append(string)
    r = {
        'ids': ids,
        'strings': strings,
    }
    return r


MASK_DIR_PATH = '/kaggle/working/predicted_masks' # change this to the path to your output mask folder
dir = MASK_DIR_PATH
res = mask2string(dir)
df = pd.DataFrame(columns=['Id', 'Expected'])
df['Id'] = res['ids']
df['Expected'] = res['strings']
df.to_csv(r'output.csv', index=False)

/kaggle/working/predicted_masks/af35b65bd9ea42cfcfedb5eb2a0e4b50.png
/kaggle/working/predicted_masks/f62f215f0da4ad3a7ab8df9da7386835.png
/kaggle/working/predicted_masks/df366e057db382b8564872a27301a654.png
/kaggle/working/predicted_masks/88e16d4ca6160127cd1d5ff99c267599.png
/kaggle/working/predicted_masks/425b976973f13dd311a65d2b46d0a608.png
/kaggle/working/predicted_masks/c695325ded465efde988dfb96d081533.png
/kaggle/working/predicted_masks/05b78a91391adc0bb223c4eaf3372eae.png
/kaggle/working/predicted_masks/6679bff55177a34fc01019eec999fd84.png
/kaggle/working/predicted_masks/6f4d4987ea3b4bae5672a230194c5a08.png
/kaggle/working/predicted_masks/e56a6d9ba9d45c3dbc695325ded465ef.png
/kaggle/working/predicted_masks/5beb48f0be11d0309d1dff09b8405734.png
/kaggle/working/predicted_masks/cdf3f33c3ca4d5060a633a8d5b2b2b55.png
/kaggle/working/predicted_masks/4c1711b62f15ec83b97bb11e8e0c4416.png
/kaggle/working/predicted_masks/63b8318ecf467d7ad048df39beb17636.png
/kaggle/working/predicted_masks/01